<a href="https://colab.research.google.com/github/mLloyd-MedPhys/AlphaParticleRadbio/blob/main/CreateTopasFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook was used to read a DICOM plan file and convert the relevant data into a text document of TOPAS parameters.

In [ ]:
#Install the Pydiocom Library
!pip install pydicom

     |████████████████████████████████| 1.9MB 6.7MB/s 


In [ ]:
#Import the pydicom library to access the data from dicom files
import pydicom

#Import maths functions from the numpy library
from numpy import pi
from numpy import arctan

#Import ET to read XML files
import xml.etree.ElementTree as ET

#Import interpolation to get MU to proton number curve
from scipy.interpolate import interp1d

import matplotlib.pyplot as plt

In [ ]:
def xml_curve(path,idx):
  '''
  Creates a curve to obtain data out from an xml file
  '''
  #Read the xml file
  tree = ET.parse(path)
  root = tree.getroot()

  #Initiate lists for x any y axes values
  energy = []
  values = []

  #Obtain curve data points from xml file. Break single string into a list
  data = root[0][idx].text.split(';')
  #Remove last blank value in list
  data.pop()

  #Iterate over every data point and add values to respective list
  for item in data:
    case = item.split(',')
    energy.append(float(case[0]))
    values.append(float(case[1]))

  #Use scipy interpolation function to create a curve
  curve = interp1d(energy,values,kind='linear')

  return curve

In [ ]:
#Establish a curve for the number of protons per MU
PNumCurve = xml_curve('/content/drive/MyDrive/004_option01_dosenormalization.xml',0)

#Establish a curve for the mean energy for a stated beam energy
real_meanE = xml_curve('/content/drive/MyDrive/004_option01_energyspectrummean.xml',0)

#Establish a curve for the energy standard deviation for a stated beam energy
energy_sigma = xml_curve('/content/drive/MyDrive/004_option01_energyspectrumsigma.xml',0)

#Establish a curve for the standard deviation of beam spread in the x direction
sigmaX = xml_curve('/content/drive/MyDrive/004_option01_spotmodelparametersx.xml',2)

#Establish a curve for the standard deviation of beam spread in the y direction
sigmaY = xml_curve('/content/drive/MyDrive/004_option01_spotmodelparametersy.xml',2)

In [ ]:
def WriteIso(iso,text):
  '''
  Determines the Isocenter position for a patient and writes the corresponding
  Topas parameter lines
  '''

  isoX = "d:Ge/IsoCenterX = " + str(iso[0]) + " mm"
  isoY = "d:Ge/IsoCenterY = " + str(iso[1]) + " mm"
  isoZ = "d:Ge/IsoCenterZ = " + str(iso[2]) + " mm"

  text.write(isoX + '\n' + isoY + '\n' + isoZ + '\n')

In [ ]:
def CreateGantry(beam_info,beam_num,text):
  '''
  Determines the position of the gantry for a given beam angle
  '''
  #Create a name for this gantry position
  name = "gantry_" + str(beam_num)

  #Read the gantry angle from DICOM file
  g_ang = int(beam_info.IonControlPointSequence[0].GantryAngle)
  rot = str(-1*g_ang)

  #Create the lines for Topas parameter file
  Par = "s:Ge/" + name + r'/Parent  = "World"'
  Type = "s:Ge/" + name + r'/Type  = "Group"'
  RotX = "d:Ge/" + name + "/RotX    = 0. deg"
  RotY = "d:Ge/" + name + "/RotY    = " + rot + " deg"
  RotZ = "d:Ge/" + name + "/RotZ    = 0. deg"
  TransX = "d:Ge/" + name + "/TransX  = 0. m"
  TransY = "d:Ge/" + name + "/TransY  = 0. m"
  TransZ = "d:Ge/" + name + "/TransZ  = 0. m"

  #Write the lines to the Topas file
  text.write("\n" + "\n" + "\n" + Par + "\n" + Type + "\n" + RotX + "\n" + RotY
             + "\n" + RotZ+ "\n" + TransX + "\n" + TransY + "\n" + 
             TransZ + "\n" + "\n")
  

In [ ]:
def SimpleSpot(CtrlIndex,sequence,SpotNum,beam,WF,SAD,text,divisor):
  '''This function takes the Ion Beam sequence data, the control point number
  and the spot number. It then writes the corresponding source information for 
  that specific spot in the Topas format to a text file'''

  if sequence.ScanSpotMetersetWeights[SpotNum] > 0:

    #Create the source name
    spotName = "/Beam" + "{:02d}".format(beam) + "Index" + "{:02d}".format(CtrlIndex) + "Spot" + "{:02d}".format(SpotNum) + "/"

    #Initiate the source as a beam
    Type = 's:So' + spotName + r'Type = "Beam"'
    text.write(Type + '\n')

    #Create the Position component
    posName = "BeamPos_Beam" + "{:02d}".format(beam) + "Index" + "{:02d}".format(CtrlIndex) + "Spot" + "{:02d}".format(SpotNum)
    Component = "s:So" + spotName + "Component = " + r'"' + posName + r'"'
    text.write(Component + '\n')

    #Make the beam a proton beam
    Particle = 's:So' + spotName + r'BeamParticle = "proton"'
    text.write(Particle + '\n')

    #Read out the nominal beam energy of the spot
    stated_Energy = sequence.NominalBeamEnergy
    mean_Energy = real_meanE(stated_Energy)
    sd_Energy = energy_sigma(stated_Energy)
    spread = 100 * sd_Energy / mean_Energy

    Energy = "d:So" + spotName + "BeamEnergy = " + str(mean_Energy) + " MeV"
    text.write(Energy + '\n')

    Energy_Spread = "u:So" + spotName + "BeamEnergySpread = " + str(spread) 
    text.write(Energy_Spread + '\n')
    

##########################################################################
    pos_dist = "s:So" + spotName + r'BeamPositionDistribution = "Gaussian"'
    pos_shape = "s:So" + spotName + r'BeamPositionCutoffShape = "Ellipse"'
    text.write(pos_dist + '\n' + pos_shape + '\n')

    xsd = sigmaX(stated_Energy)
    x_w = 4*xsd
    SpreadX = "d:So" + spotName + "BeamPositionSpreadX = " +str(xsd) + " mm"
    CutX = "d:So" + spotName + "BeamPositionCutoffX = " + str(x_w) + " mm"
    text.write(SpreadX + '\n' + CutX + '\n')

    ysd = sigmaY(stated_Energy)
    y_w = 4*ysd
    SpreadY = "d:So" + spotName + "BeamPositionSpreadY = " + str(ysd) + " mm"
    CutY = "d:So" + spotName + "BeamPositionCutoffY = " + str(y_w) + " mm"
    text.write(SpreadY + '\n' + CutY + '\n')

    Angle = "s:So" + spotName + r'BeamAngularDistribution = "None"'
    text.write(Angle + '\n')
###############################################################################
    
    #Find the number of histories for the spot
    mon_units = sequence.ScanSpotMetersetWeights[SpotNum] * WF
    p_perMU = float(PNumCurve(sequence.NominalBeamEnergy))
    num_pro = round((p_perMU * mon_units)/divisor)
    Hist = "i:So" + spotName + "NumberOfHistoriesInRun = " + str(num_pro)
    text.write(Hist + '\n')

    #Create a line break between source information and the source geometry
    text.write('\n')



    #Attach the source to the IEC gantry coordinate group
    Parent = "s:Ge/" + posName + r'/Parent = "' + "gantry_" + str(beam) + r'"'
    text.write(Parent + '\n')

    #Designate the type as group 
    Group = "s:Ge/" + posName + r'/Type= "Group"'
    text.write(Group + '\n')
   
    #Simulate angular deflection due to magnet. Deviation in y direction due to
    #a rotation around x.
    thetaX = pi - arctan(sequence.ScanSpotPositionMap[2 * SpotNum +1]/SAD)

    #Simulate anguar deflection due to magnet. Deviation in x direction due to
    #a rotation around y
    thetaY = -1 * arctan(sequence.ScanSpotPositionMap[2 * SpotNum]/SAD)

    #Add rotations to Topas file
    RotX = "d:Ge/" + posName + "/RotX = " + str(thetaX) + " rad"
    RotY = "d:Ge/" + posName + "/RotY = " + str(thetaY) + " rad"
    text.write(RotX + '\n' + RotY + '\n')

    #Want particles to be produced at a point above the isocentre: (0,0,SAD).
    trX = 0
    trY = 0
    trZ = SAD

    #Combine results for position of Topas source
    TransX = "d:Ge/" + posName + "/TransX = " + str(trX) + " mm"
    TransY = "d:Ge/" + posName + "/TransY = " + str(trY) + " mm"
    TransZ = "d:Ge/" + posName + "/TransZ = " + str(trZ) + " mm"
    text.write(TransX + '\n'+TransY + '\n'+TransZ + '\n')


    #Provide a larger break between different sources
    text.write('\n \n \n')

In [ ]:
def WriteTopas(fileName, plan, divisor):
  """Takes in a Dicom planning file (plan) and writes Topas geometry and source 
  parameters to a file called fileName.txt  """

  #Open the text file
  text = open(fileName, "wt")

  #Assume only one fraction group is used
  FG_Data = plan.FractionGroupSequence[0]

  #Find the isocentre and record the parameters
  iso = plan.IonBeamSequence[0].IonControlPointSequence[0].IsocenterPosition
  WriteIso(iso,text)

  tot = 0


  #Iterate over each beam. For total use FG_Data.NumberOfBeams
  for beam in range(0,1):

    #Isolate plan information for the desired beam
    IonBeam = plan.IonBeamSequence[beam]

    #Write Topas parameters for gantry geometry
    gan_name = CreateGantry(IonBeam,beam,text)

    #Determine the weighting factor(WF) of this beam
    BeamMeterset = FG_Data.ReferencedBeamSequence[beam].BeamMeterset
    FCMetersetWeight = IonBeam.FinalCumulativeMetersetWeight
    WF = BeamMeterset / FCMetersetWeight

    #Read the number of Control points from the Dicom
    CPointRange = int(IonBeam.NumberOfControlPoints)



    #Iterate over all control points
    for CtrlIndex in range(CPointRange):
      
      #Specify data for this control point
      sequence = IonBeam.IonControlPointSequence[CtrlIndex]

      #Find the number of spots for this control point
      SpotRange = int(sequence.NumberOfScanSpotPositions)



      #Iterate over all of the spots in the current control point
      for SpotNum in range(SpotRange):

        #Run the function to write down the spot source data
        SimpleSpot(CtrlIndex,sequence,SpotNum,beam,WF,2280,text,divisor)

        mon_units = sequence.ScanSpotMetersetWeights[SpotNum]*WF
        p_perMU = float(PNumCurve(sequence.NominalBeamEnergy))
        num_pro = round(p_perMU * mon_units/divisor)
        tot = tot + num_pro




  #Close the file
  text.close()

  return tot

In [ ]:
#Read the plan dicom file
rn = pydicom.dcmread("/content/RN.1.2.246.352.71.5.71129641657.2052905.20210520131306.dcm")

In [ ]:
#Write the Topas parameter files
WriteTopas("RedoCyl.txt",rn,1000)

21656731